<a href="https://colab.research.google.com/github/migostro/laboratorio-de-visao/blob/main/3_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-image==0.19.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from google.colab import files, drive
import os
from skimage import io, color
from skimage import segmentation, measure

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

/usr/local/lib/python3.7/dist-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


# Helper Functions

In [ ]:
# list_filepaths(): list filepath for every filepath on a root folder
# pre-condition: (root path, empty list)
# post-condition: list with every filepath on root path
def list_filepaths(path, file_extension, filepaths = []):
    if os.path.isdir(path):
        for filename in os.listdir(path):
            filepath = os.path.join(path, filename)
            if os.path.isfile(filepath) and file_extension in filename: filepaths.append(filepath) # Adiciona apenas caminhos que são de arquivos (que no nosso caso são imagens)
            else: list_filepaths(filepath, file_extension, filepaths)

    return filepaths

# Pipeline

In [ ]:
# 0 FILE READING

drive.mount('/content/drive')
root = '/content/drive/MyDrive/Colab/MAC0417/Trabalho'
#root = '/content/drive/MyDrive/ime/mac0417/ep'
originalPath = root + "/originalDataSet_resized"
segmented_path = root + "/segmented_resized"
groundTruthDataPath = root + "/groundTruth_resized"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def create_list_segmented(original_paths):
    new_paths = []
    for path in original_paths:
        new_path = path.split('/')
        new_path[-6] = 'segmented_resized'
        new_format_photo = new_path[-1].split('.')
        new_format_photo[-1] = 'png'

        new_path[-1] = new_format_photo[0] + '.' + 'png'

        aux_new_path = '/'
        for i in new_path:
            aux_new_path = os.path.join(aux_new_path, i)
        new_paths.append(aux_new_path)
    return new_paths


In [ ]:
originalPaths = list_filepaths(originalPath, 'JPG', [])
segmentedPaths = create_list_segmented(originalPaths)

print(f'# of images @ {originalPath}: {len(originalPaths)}')
print(f'# of images @ {segmented_path}: {len(segmentedPaths)}')
print(originalPaths)
print(segmentedPaths)

# of images @ /content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized: 1080
# of images @ /content/drive/MyDrive/ime/mac0417/ep/segmented_resized: 1080
['/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0856.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0857.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0858.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0859.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0860.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0861.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/borrachas/IMG_0862.JPG', '/content/drive/MyDrive/ime/mac0417/ep/originalDataSet_resized/dia/interior/branco/bo

In [ ]:
classes = ['borrachas', 'cartas', 'celulares', 'conchas', 'copos', 'dados', 'estatuas', 'lapis', 'tubos', 'vasos']
num_samples = len(originalPaths)
num_features = 9
num_classes = len(classes)
X = np.zeros([num_samples, num_features])
Y = np.zeros([num_samples, num_classes])

# https://scikit-learn.org/stable/modules/feature_extraction.html
for i in range(num_samples):
    for j in range(num_classes):
        if originalPaths[i].split('/')[-2] == classes[j]:
            Y[i, j] = 1

## Creating the feature vector

In [ ]:
# create feature vector
for i in range(num_samples):
    print(f'[{i}]')
    img_original = io.imread(originalPaths[i])
    img_segmented = io.imread(segmentedPaths[i])
    cut = np.multiply(img_original, img_segmented)

    label_img = measure.label(cut)
    regions = measure.regionprops(label_img)

    if(len(regions) > 1): # isolate_largest_area
        largest_region = max(regions, key=lambda region: region.area)

    if len(regions) == 1:
        largest_region = regions[0]
        
    X[i, 0] = largest_region.area
    X[i, 1] = largest_region.solidity
    X[i, 2] = largest_region.eccentricity
    X[i, 3] = largest_region.feret_diameter_max
    X[i, 4] = largest_region.perimeter
    X[i, 5] = largest_region.orientation
    X[i, 6] = largest_region.extent
    X[i, 7] = largest_region.axis_major_length
    X[i, 8] = largest_region.axis_minor_length

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
[30]
[31]
[32]
[33]
[34]
[35]
[36]
[37]
[38]
[39]
[40]
[41]
[42]
[43]
[44]
[45]
[46]
[47]
[48]
[49]
[50]
[51]
[52]
[53]
[54]
[55]
[56]
[57]
[58]
[59]
[60]
[61]
[62]
[63]
[64]
[65]
[66]
[67]
[68]
[69]
[70]
[71]
[72]
[73]
[74]
[75]
[76]
[77]
[78]
[79]
[80]
[81]
[82]
[83]
[84]
[85]
[86]
[87]
[88]
[89]
[90]
[91]
[92]
[93]
[94]
[95]
[96]
[97]
[98]
[99]
[100]
[101]
[102]
[103]
[104]
[105]
[106]
[107]
[108]
[109]
[110]
[111]
[112]
[113]
[114]
[115]
[116]
[117]
[118]
[119]
[120]
[121]
[122]
[123]
[124]
[125]
[126]
[127]
[128]
[129]
[130]
[131]
[132]
[133]
[134]
[135]
[136]
[137]
[138]
[139]
[140]
[141]
[142]
[143]
[144]
[145]
[146]
[147]
[148]
[149]
[150]
[151]
[152]
[153]
[154]
[155]
[156]
[157]
[158]
[159]
[160]
[161]
[162]
[163]
[164]
[165]
[166]
[167]
[168]
[169]
[170]
[171]
[172]
[173]
[174]
[175]
[176]
[177]
[178]
[179]
[180]
[181]
[182]
[183]
[184]


In [ ]:
print(X.shape, Y.shape)

(1080, 9) (1080, 10)


## Finding a good combination of alpha, number of layers and epislom
Here we test all combinations between some values for each variable and after choosing the best combination that had the score.

In [ ]:
alphas = np.arange(0.1, 1.1, 0.1)
epsilons = np.array([1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8])
layers = np.arange(10, 101, 10)

print(alphas)
print(epsilons)
print(layers)

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
[1.e-03 1.e-04 1.e-05 1.e-06 1.e-07 1.e-08]
[ 10  20  30  40  50  60  70  80  90 100]


In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

alphas = np.arange(0.4, 1.1, 0.1)
epsilons = np.array([1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8])
layers = np.arange(10, 101, 10)

cases = []
for a in range(len(alphas)):
    for e in range(len(epsilons)):
        for l in range(len(layers)):
            
            # create the classifier
            classifier = MLPClassifier(solver='adam', alpha=alphas[a], epsilon=epsilons[e], hidden_layer_sizes=(layers[l]), max_iter=10000, n_iter_no_change=50, random_state=1)

            # train the model
            classifier.fit(X_train, Y_train)

            # calculate the avg score
            score = classifier.score(X_test, Y_test)

            vetor = []

            vetor.append(alphas[a])
            vetor.append(epsilons[e])
            vetor.append(layers[l])
            vetor.append(score)

            cases.append(vetor)
    print(a)
            


0
1
2
3
4
5
6


In [ ]:
print(cases)

[[0.4, 0.001, 10, 0.022222222222222223], [0.4, 0.001, 20, 0.07777777777777778], [0.4, 0.001, 30, 0.11481481481481481], [0.4, 0.001, 40, 0.02962962962962963], [0.4, 0.001, 50, 0.07037037037037037], [0.4, 0.001, 60, 0.11481481481481481], [0.4, 0.001, 70, 0.08888888888888889], [0.4, 0.001, 80, 0.09259259259259259], [0.4, 0.001, 90, 0.08148148148148149], [0.4, 0.001, 100, 0.1259259259259259], [0.4, 0.0001, 10, 0.011111111111111112], [0.4, 0.0001, 20, 0.09259259259259259], [0.4, 0.0001, 30, 0.12962962962962962], [0.4, 0.0001, 40, 0.07407407407407407], [0.4, 0.0001, 50, 0.06666666666666667], [0.4, 0.0001, 60, 0.07037037037037037], [0.4, 0.0001, 70, 0.1], [0.4, 0.0001, 80, 0.12222222222222222], [0.4, 0.0001, 90, 0.05925925925925926], [0.4, 0.0001, 100, 0.13703703703703704], [0.4, 1e-05, 10, 0.040740740740740744], [0.4, 1e-05, 20, 0.07407407407407407], [0.4, 1e-05, 30, 0.08888888888888889], [0.4, 1e-05, 40, 0.025925925925925925], [0.4, 1e-05, 50, 0.08148148148148149], [0.4, 1e-05, 60, 0.114814

In [ ]:
maior = 0
for i in range(len(cases)):
    if cases[i][3] > cases[maior][3]:
        maior = i
print(cases[maior][3])
print(cases[maior][:])

0.17037037037037037
[0.4, 1e-05, 80, 0.17037037037037037]


## Results
No final, nosso score geral foi de aproximadamente 16% de acerto, não sendo um resultado bom. Acreditamos que não tenha sido bom, já que, ao se plotar alguns dados juntos, não foi possível ver uma distinção clara das classes, tendo agrupamentos de classes muito próximos um dos outros, dificultando muito a classificação. E em segundo, acreditamos que o fator anterior foi amplificado em imagens que não tiveram uma segmentação muito boa, dado a dificuladade do problema.

In [ ]:
a = cases[maior][0]
e = cases[maior][1]
l = cases[maior][2]

best_clf = MLPClassifier(alpha=a, epsilon=e, hidden_layer_sizes=(l), max_iter=10000, n_iter_no_change=50, random_state=1)
best_clf.fit(X_train, Y_train)

print("Score using the train set", best_clf.score(X_train, Y_train))
print("Score using the test", best_clf.score(X_test, Y_test))
print("General score", best_clf.score(X, Y))

Score using the train set 0.1617283950617284
Score using the test 0.17037037037037037
General score 0.1638888888888889
